# LLM 20 Questions Baseline

You can change models and prompts conveniently since I used `tokenizer.apply_chat_template` to apply special tokens automatically. 


Supported models:
- `LLAMA3 variants`
- `Phi-3 variants`
- `Qwen-2 variants`

## Prerequisites
Set accelerator to GPU T4

### Select Model

Find your desired model from [HuggingFace Model Hub](https://huggingface.co/models) and use the model name in the next command.

Supported models:
- `LLAMA3 variants`
- `Phi-3 variants`
- `Qwen-2 variants`

In [3]:
from huggingface_hub import login

login()

In [4]:
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"

### Download Model via HuggingFace
To reduce disk usage, download model in `/tmp/model`

In [6]:
import os
os.listdir('./')

['Documents',
 'FEAR_w_videos.pptx',
 'FEDERALWORKSHEETS_2024-02-18_1708313627322.pdf',
 'FTF_2024-02-18_1708313625683.pdf',
 'Game AI - Coordinated Movement NOTES.pdf',
 'Game AI - Learning - Decision Trees NOTES.pdf',
 'Game AI - Learning - NGrams_NOTES.pdf',
 'Game AI - Learning - Q-Learning NOTES.pdf',
 'Game AI - PCG - Intro NOTES.pdf',
 'Game AI - PCG - Object Placement NOTES.pdf',
 'Game AI Fuzzy Logic.pdf',
 'Game AI Goal Oriented Behavior - Part I NOTES.pdf',
 'Game AI Goal Oriented Behavior - Part II NOTES.pdf',
 'Game AI RBS Intro.pdf',
 'GameAI AgentMovement-Intro and Kinematics (1).pdf',
 'GameAI AgentMovement-Intro and Kinematics REDUCED (1).pdf',
 'GameAI AgentMovement-Intro and Kinematics REDUCED.pdf',
 'GameAI AgentMovement-Intro and Kinematics.pdf',
 'GameAI Behavior Trees.pdf',
 'GameAI Computational Geometry.pdf',
 'GameAI Intro.pdf',
 'GameAI Path Planning Intro and Grid.pdf',
 'GameAI Path Planning Path Networks.pdf',
 'GameAI Path Planning Search Algorithms.pdf',

In [5]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("./")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id=repo_id,
    ignore_patterns="original*",
    local_dir=g_model_path,
    token=globals().get("HF_TOKEN", None)
)

PermissionError: [WinError 5] Access is denied: '.\\Documents'

In [5]:
!ls -l /tmp/model

total 15693180
-rw-r--r-- 1 root root       7801 Aug  1 02:43 LICENSE
-rw-r--r-- 1 root root      38906 Aug  1 02:43 README.md
-rw-r--r-- 1 root root       4696 Aug  1 02:43 USE_POLICY.md
-rw-r--r-- 1 root root        654 Aug  1 02:43 config.json
-rw-r--r-- 1 root root        187 Aug  1 02:43 generation_config.json
-rw-r--r-- 1 root root 4976698672 Aug  1 02:46 model-00001-of-00004.safetensors
-rw-r--r-- 1 root root 4999802720 Aug  1 02:46 model-00002-of-00004.safetensors
-rw-r--r-- 1 root root 4915916176 Aug  1 02:46 model-00003-of-00004.safetensors
-rw-r--r-- 1 root root 1168138808 Aug  1 02:44 model-00004-of-00004.safetensors
-rw-r--r-- 1 root root      23950 Aug  1 02:43 model.safetensors.index.json
-rw-r--r-- 1 root root         73 Aug  1 02:43 special_tokens_map.json
-rw-r--r-- 1 root root    9085698 Aug  1 02:43 tokenizer.json
-rw-r--r-- 1 root root      50977 Aug  1 02:43 tokenizer_config.json


### Save quantized model
Now, load downloaded model on memory with quantization.  
This will save storage.


Moreover, since the saved model has already been quantized, we do not need `bitsandbytes` package in `main.py`

In [6]:
# load model on memory
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

downloaded_model = "/tmp/model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    downloaded_model,
    quantization_config = bnb_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
)

tokenizer = AutoTokenizer.from_pretrained(downloaded_model)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# save model in submission directory
model.save_pretrained("/kaggle/working/submission/model")
tokenizer.save_pretrained("/kaggle/working/submission/model")

('/kaggle/working/submission/model/tokenizer_config.json',
 '/kaggle/working/submission/model/special_tokens_map.json',
 '/kaggle/working/submission/model/tokenizer.json')

In [8]:
# unload model from memory
import gc, torch
del model, tokenizer
gc.collect()
torch.cuda.empty_cache()

## Agent

### Prompts

Prompts are reffered from the [Anthropic Prompt Library](https://docs.anthropic.com/en/prompt-library/library)

Prompts are consisted of 2 parts:
- `system_prompt`: This is the first question to determine category.
- `chat_history`: This is the chat history to provide context for the model.

In [9]:
%%writefile submission/prompts.py

def asker_prompt(obs):
    message = []
    
    # System prompt
    ask_prompt = f"""You are a helpful AI assistant with expertise in playing 20 questions game.
Your task is to ask questions to the user to guess the word the user is thinking of.
Narrow down the possibilities by asking yes/no questions.
Think step by step and try to ask the most informative questions.
\n"""

    message.append({"role": "system", "content": ask_prompt})

    # Chat history
    for q, a in zip(obs.questions, obs.answers):
        message.append({"role": "assistant", "content": q})
        message.append({"role": "user", "content": a})

    return message


def guesser_prompt(obs):
    message = []
    
    # System prompt
    guess_prompt = f"""You are a helpful AI assistant with expertise in playing 20 questions game.
Your task is to guess the word the user is thinking of.
Think step by step.
\n"""

    # Chat history
    chat_history = ""
    for q, a in zip(obs.questions, obs.answers):
        chat_history += f"""Question: {q}\nAnswer: {a}\n"""
    prompt = (
            guess_prompt + f"""so far, the current state of the game is as following:\n{chat_history}
        based on the conversation, can you guess the word, please give only the word, no verbosity around"""
    )
    
    
    message.append({"role": "system", "content": prompt})
    
    return message


def answerer_prompt(obs):
    message = []
    
    # System prompt
    prompt = f"""You are a helpful AI assistant with expertise in playing 20 questions game.
Your task is to answer the questions of the user to help him guess the word you're thinking of.
Your answers must be 'yes' or 'no'.
The keyword is: "{obs.keyword}", it is of category: "{obs.category}"
Provide accurate answers to help the user to guess the keyword.
"""

    message.append({"role": "system", "content": prompt})
    
    # Chat history
    message.append({"role": "user", "content": obs.questions[0]})
    
    if len(obs.answers)>=1:
        for q, a in zip(obs.questions[1:], obs.answers):
            message.append({"role": "assistant", "content": a})
            message.append({"role": "user", "content": q})
    
    return message



Writing submission/prompts.py


### Agent

To add more LLM models, add end-of-turn token to terminators list.

In [10]:
%%writefile submission/main.py

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

from prompts import *

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    MODEL_PATH = os.path.join(KAGGLE_AGENT_PATH, "model")
else:
    MODEL_PATH = "/kaggle/working/submission/model"


model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# specify end-of-turn tokens for your desired model
terminators = [tokenizer.eos_token_id]

# Additional potential end-of-turn token
# llama3, phi3, gwen2 by order
potential_terminators = ["<|eot_id|>", "<|end|>", "<end_of_turn>"]

for token in potential_terminators:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id is not None:
        terminators.append(token_id)

def generate_response(chat):
    inputs = tokenizer.apply_chat_template(chat, add_generation_prompt=True, return_tensors="pt").to(model.device)
    outputs = model.generate(inputs, max_new_tokens=32, pad_token_id=tokenizer.eos_token_id, eos_token_id=terminators)
    response = outputs[0][inputs.shape[-1]:]
    out = tokenizer.decode(response, skip_special_tokens=True)

    return out


class Robot:
    def __init__(self):
        pass

    def on(self, mode, obs):
        assert mode in [
            "asking", "guessing", "answering",
        ], "mode can only take one of these values: asking, answering, guessing"
        if mode == "asking":
            # launch the asker role
            output = self.asker(obs)
        if mode == "answering":
            # launch the answerer role
            output = self.answerer(obs)
            if "yes" in output.lower():
                output = "yes"
            elif "no" in output.lower():
                output = "no"
            if "yes" not in output.lower() and "no" not in output.lower():
                output = "yes"
        if mode == "guessing":
            # launch the guesser role
            output = self.guesser(obs)
        return output

    def asker(self, obs):
        
        input = asker_prompt(obs)
        output = generate_response(input)
        
        return output

    def guesser(self, obs):
        input = guesser_prompt(obs)
        output = generate_response(input)
        
        return output

    def answerer(self, obs):
        input = answerer_prompt(obs)
        output = generate_response(input)

        return output


robot = Robot()


def agent(obs, cfg):

    if obs.turnType == "ask":
        response = robot.on(mode="asking", obs=obs)

    elif obs.turnType == "guess":
        response = robot.on(mode="guessing", obs=obs)

    elif obs.turnType == "answer":
        response = robot.on(mode="answering", obs=obs)

    if response == None or len(response) <= 1:
        response = "yes"

    return response


Writing submission/main.py


## Simulation

### Install pygame

In [11]:
!pip install pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 9.3 MB/s eta 0:00:000:00:010:01


In [12]:
%%writefile dumb.py

def dumb_agent(obs, cfg):
    
    # if agent is guesser and turnType is "ask"
    if obs.turnType == "ask":
        response = "Is it a duck?"
    # if agent is guesser and turnType is "guess"
    elif obs.turnType == "guess":
        response = "duck"
    # if agent is the answerer
    elif obs.turnType == "answer":
        response = "no"
    
    return response

Writing dumb.py


In [13]:
%%time

from kaggle_environments import make
env = make("llm_20_questions", debug=True)
game_output = env.run(agents=["submission/main.py", "submission/main.py", "dumb.py", "dumb.py"])

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
2024-08-01 02:48:11.111513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 02:48:11.111641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 02:48:11.240218: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_c

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 1min 31s, sys: 5.6 s, total: 1min 37s
Wall time: 1min 42s


In [18]:
env.render(mode="ipython", width=600, height=700)

## Submit Agent

In [15]:
!apt install pigz pv > /dev/null

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


4.73GiB 0:01:35 [50.8MiB/s] [   <=>                                            ]
